In [1]:
import numpy as np

class SVM:
    def __init__(self, C=1.0, tol=1e-3, max_iter=100):
        # SVM hyperparameters
        self.C = C  # Regularization parameter
        self.tol = tol  # Tolerance for numerical stability
        self.max_iter = max_iter  # Maximum number of iterations for the SMO algorithm

    def fit(self, X, y):
        # Training the SVM using the Sequential Minimal Optimization (SMO) algorithm
        self.X = X  # Training data
        self.y = y  # Labels
        self.alpha = np.zeros(len(X))  # Lagrange multipliers
        self.b = 0.0  # Intercept term

        for _ in range(self.max_iter):
            num_changed_alphas = 0

            # Iterate over all training examples
            for i in range(len(X)):
                # Compute the decision function value for the current example
                E_i = self.decision_function(X[i]) - y[i]

                # Check if the example violates the KKT conditions
                if ((y[i] * E_i < -self.tol and self.alpha[i] < self.C) or
                        (y[i] * E_i > self.tol and self.alpha[i] > 0)):
                    # Select a second example j different from i
                    j = self.select_random_index(i)
                    E_j = self.decision_function(X[j]) - y[j]

                    # Save old alpha values for later comparison
                    alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]

                    # Compute the bounds for alpha[j] to satisfy KKT conditions
                    if y[i] != y[j]:
                        L = max(0, self.alpha[j] - self.alpha[i])
                        H = min(self.C, self.C + self.alpha[j] - self.alpha[i])
                    else:
                        L = max(0, self.alpha[i] + self.alpha[j] - self.C)
                        H = min(self.C, self.alpha[i] + self.alpha[j])

                    # Check if the bounds are equal, continue to the next iteration
                    if L == H:
                        continue

                    # Compute the second derivative of the objective function
                    eta = 2 * np.dot(X[i], X[j]) - np.dot(X[i], X[i]) - np.dot(X[j], X[j])

                    # If eta is non-positive, continue to the next iteration
                    if eta >= 0:
                        continue

                    # Update alpha[j]
                    self.alpha[j] -= (y[j] * (E_i - E_j)) / eta
                    self.alpha[j] = np.clip(self.alpha[j], L, H)

                    # If the change in alpha[j] is small, continue to the next iteration
                    if abs(self.alpha[j] - alpha_j_old) < 1e-5:
                        continue

                    # Update alpha[i]
                    self.alpha[i] += y[i] * y[j] * (alpha_j_old - self.alpha[j])

                    # Update the intercept term
                    b1 = self.b - E_i - y[i] * (self.alpha[i] - alpha_i_old) * np.dot(X[i], X[i]) - \
                         y[j] * (self.alpha[j] - alpha_j_old) * np.dot(X[i], X[j])
                    b2 = self.b - E_j - y[i] * (self.alpha[i] - alpha_i_old) * np.dot(X[i], X[j]) - \
                         y[j] * (self.alpha[j] - alpha_j_old) * np.dot(X[j], X[j])

                    if 0 < self.alpha[i] < self.C:
                        self.b = b1
                    elif 0 < self.alpha[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2

                    num_changed_alphas += 1

            # If no alphas were changed in this iteration, exit the loop
            if num_changed_alphas == 0:
                break

    def decision_function(self, x):
        # Decision function for predicting the class of a sample
        return np.dot(self.alpha * self.y, np.dot(self.X, x)) - self.b

    def predict(self, X):
        # Make predictions on input data
        return np.sign(np.array([self.decision_function(x) for x in X]))

    def select_random_index(self, i):
        # Select a random index different from the given index i
        j = i
        while j == i:
            j = np.random.randint(len(self.X))
        return j


# Example usage:
# Assuming X_train and y_train are your training data and labels
# X_test is your testing data

# Generate some random data for testing
np.random.seed(42)
X_train = np.random.randn(100, 2)
y_train = np.where(X_train[:, 0] + X_train[:, 1] > 0, 1, -1)

svm = SVM(C=1.0)
svm.fit(X_train, y_train)

# Make predictions on the training data
y_pred_train = svm.predict(X_train)

# Print accuracy on the training data
accuracy_train = np.mean(y_pred_train == y_train)
print(f"Training Accuracy: {accuracy_train}")

# Assuming you have some testing data X_test
# Make predictions on the test data
y_pred_test = svm.predict(X_test)

# Print predictions on the test data
print("Predictions on Test Data:", y_pred_test)


Training Accuracy: 0.44


NameError: name 'X_test' is not defined

In [ ]:
import numpy as np

class SVM_classifier():

  # Initiating the hyperparameters
  def __init__(self, learning_rate, no_of_iterations, lambda_parameter):
    self.learning_rate = learning_rate
    self.no_of_iterations = no_of_iterations
    self.lambda_parameter = lambda_parameter

  # Fitting the dataset to SVM Classifier
  def fit(self, X, Y):
    # m  --> number of data points (rows)
    # n  --> number of input features (columns)
    self.m, self.n = X.shape

    # Initiating the weight value and bias value
    self.w = np.zeros(self.n)
    self.b = 0
    self.X = X
    self.Y = Y

    # Implementing Gradient Descent algorithm for optimization
    for i in range(self.no_of_iterations):
      self.update_weights()

  # Function for updating the weight and bias value
  def update_weights(self):
    # Label encoding: Convert Y values to -1 for <= 0 and 1 for > 0
    y_label = np.where(self.Y <= 0, -1, 1)

    # Gradients (dw, db) for each data point
    for index, x_i in enumerate(self.X):
      # Condition for checking if the point is correctly classified
      condition = y_label[index] * (np.dot(x_i, self.w) - self.b) >= 1

      if condition:
        # If correctly classified, update gradients without considering the data point
        dw = 2 * self.lambda_parameter * self.w
        db = 0
      else:
        # If incorrectly classified, update gradients considering the data point
        dw = 2 * self.lambda_parameter * self.w - np.dot(x_i, y_label[index])
        db = y_label[index]

      # Update weights and bias using the gradients and learning rate
      self.w = self.w - self.learning_rate * dw
      self.b = self.b - self.learning_rate * db

  # Predict the label for a given input value
  def predict(self, X):
    # Calculate the output based on the learned weights and bias
    output = np.dot(X, self.w) - self.b

    # Convert the output to predicted labels (0 or 1) using sign function
    predicted_labels = np.sign(output)

    # Convert predicted labels to 0 if <= -1, and 1 otherwise
    y_hat = np.where(predicted_labels <= -1, 0, 1)

    return y_hat
